# CS4248 Project
(DON'T CLICK RUN ALL! Lemmatization takes tooo long and pls don't run it again)

In [30]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# %cd "/content/drive/My Drive/CS4248 Project"
# !cd "/content/drive/My Drive/CS4248 Project"

%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
X_train = train['text']
y_train = train['class']

In [33]:
train.head()

,class,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [34]:
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])
X_test = test['text']
y_test = test['class']

In [35]:
test.head()

,class,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


## Baseline Models

In [36]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,1), max_features = 10000)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = tfidf(X_train, X_test)

## Preprocessing

### Lemmatization with POS Tagging
(DON'T RUN AGAIN! It takes tooooo long)

In [37]:
# # WORDNET LEMMATIZER (with appropriate pos tags)
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
# 
# lemmatizer = WordNetLemmatizer()
# 
# # Define function to lemmatize each word with its POS tag
# 
# # POS_TAGGER_FUNCTION : TYPE 1
# def pos_tagger(nltk_tag):
#     if nltk_tag.startswith('J'):
#         return wordnet.ADJ
#     elif nltk_tag.startswith('V'):
#         return wordnet.VERB
#     elif nltk_tag.startswith('N'):
#         return wordnet.NOUN
#     elif nltk_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return None
# 
# def lemmatize_with_pos(tokens):
#   # tokenize the sentence and find the POS tag for each token
#   pos_tagged = nltk.pos_tag(tokens)
# 
#   # print(pos_tagged)
# 
#   wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
#   # print(wordnet_tagged)
# 
#   lemmatized_sentence = []
#   for word, tag in wordnet_tagged:
#       if tag is None:
#           # if there is no available tag, append the token as is
#           lemmatized_sentence.append(word)
#       else:
#           # else use the tag to lemmatize the token
#           lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
#   # lemmatized_sentence = " ".join(lemmatized_sentence)
# 
#   # print(lemmatized_sentence)
#   return lemmatized_sentence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [38]:
# # Start preprocessing
# 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from tqdm import tqdm
from spellchecker import SpellChecker
# 
# punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
# 
# spell = SpellChecker()
# 
# def preprocessing(data):
#   data_clean = []
#   for sentence in tqdm(data):
# 
#     # Tokenization
#     tokens = word_tokenize(sentence)
# 
#     # Remove punctuation and number
#     tokens = [w for w in tokens if (not w in punc) and (not w.isdigit())]
# 
#     # Spell check
#     # Taking too long time for each sentence, not practical to be used
#     # tokens = [spell.correction(w) for w in tokens]
#     # tokens = [w for w in tokens if w is not None and len(w) > 0]
# 
#     # Lemmatization based on tagging
#     tokens = lemmatize_with_pos(tokens)
# 
#     data_clean.append((' ').join(tokens))
#   return data_clean
# 
# X_train_clean = preprocessing(X_train)
# X_test_clean = preprocessing(X_test)

[nltk_data] Downloading package punkt to /Users/stella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stella/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
# import csv
# import pandas as pd
# 
# def write_to_csv(X_train_clean, X_test_clean):
#     with open('raw_data/X_train_clean.csv', 'w', newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         for doc in X_train_clean:
#             writer.writerow([doc])
# 
#     with open('raw_data/X_test_clean.csv', 'w', newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         for doc in X_test_clean:
#             writer.writerow([doc])
# 
# # Uncomment if you want to save X_train_clean and X_test_clean
# # write_to_csv(X_train_clean, X_test_clean)

In [40]:
def read_from_file():
    a_df = pd.read_csv('raw_data/X_train_clean.csv', header=None, dtype=str)
    a = a_df[0].values
    
    b_df = pd.read_csv('raw_data/X_test_clean.csv', header=None, dtype=str)
    b = b_df[0].values
    return a, b

# Uncomment if you want to read from file
a, b = read_from_file()
X_train_clean, X_test_clean = a, b

In [41]:
X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

### Word Embeddings
#### GloVe
- Download from https://nlp.stanford.edu/projects/glove/
- Used Wikipedia 2014 + Gigaword 5

In [42]:
def load_glove_model(glove_file_path):
    glove_model = {}
    with open(glove_file_path, 'r', encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            # embedding = np.array([float(val) for val in split_line[1:]])
            try:
                coefs = np.asarray(split_line[1:], dtype='float32')
            except ValueError:
                pass
            glove_model[word] = coefs
    return glove_model

glove_model = load_glove_model("glove.6B/glove.6B.300d.txt")

In [43]:
def preprocess_text(text):
    return word_tokenize(text.lower())

X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

In [44]:
def document_vector(doc, model):
    # Filter out words that are not in the embedding
    embeddings = [model[word] for word in doc if word in model]
    if not embeddings:
        # If no words in the document are in the model, return a vector of zeros
        return np.zeros(next(iter(model.values())).shape)
    # Average the embeddings
    return np.mean(embeddings, axis=0)

X_train_vectors = np.array([document_vector(doc, glove_model) for doc in X_train_processed])
X_test_vectors = np.array([document_vector(doc, glove_model) for doc in X_test_processed])

In [45]:
# TODO!: delete
# def text_to_features(tokens, glove_model):
#     vectors = [glove_model[token] for token in tokens if token in glove_model]
#     if vectors:
#         return np.mean(vectors, axis=0) # Return the mean of the vectors
#     else:
#         return np.zeros(300)  # Return a zero vector if no words are found
# 
# # Convert X_train_clean to a pandas Series
# X_train_clean_series = pd.Series(X_train_clean)
# # Convert text to features using a predefined function (e.g., using GloVe embeddings)
# glove_embedding_features = X_train_clean_series.apply(lambda x: pd.Series(text_to_features(x, glove_model)))
# glove_embedding_features.columns = [f'Feature_{i}' for i in range(300)]
# 
# # Use the same function to convert X_test_clean to a pandas Series
# X_test_clean_series = pd.Series(X_test_clean)
# glove_embedding_features_test = X_test_clean_series.apply(lambda x: pd.Series(text_to_features(x, glove_model)))
# glove_embedding_features_test.columns = [f'Feature_{i}' for i in range(300)]

## SVM

In [49]:
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def SVM(X_train, y_train, X_test, y_test):
    le = LabelEncoder()
    y_train_en = le.fit_transform(y_train)
    y_test_en = le.transform(y_test)
    model = svm.SVC(kernel='linear', class_weight='balanced', probability=True) # Best Model chosen by Grid Search
    model.fit(X_train, y_train_en)
    predictions_y = model.predict(X_test)
    print("SVM Accuracy Score -> ", accuracy_score(predictions_y, y_test_en) * 100)
    print(classification_report(y_test_en, predictions_y))
    cm = confusion_matrix(y_test_en, predictions_y)
    print("Confusion Matrix:")
    print(cm)

### Approach 1: tfidf 

In [17]:
_ = SVM(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

[LibSVM]....*...*
optimization finished, #iter = 7139
obj = -789.401718, rho = 0.700044
nSV = 2220, nBSV = 397
Total nSV = 2220
....*...*
optimization finished, #iter = 7298
obj = -795.939370, rho = 0.746327
nSV = 2270, nBSV = 409
Total nSV = 2270
....*...*
optimization finished, #iter = 7141
obj = -787.459236, rho = 0.742474
nSV = 2204, nBSV = 381
Total nSV = 2204
....*..*
optimization finished, #iter = 6952
obj = -790.666170, rho = 0.684879
nSV = 2243, nBSV = 421
Total nSV = 2243
....*...*
optimization finished, #iter = 7207
obj = -806.421280, rho = 0.734366
nSV = 2247, nBSV = 406
Total nSV = 2247
.....*...*
optimization finished, #iter = 8148
obj = -912.494670, rho = -0.746157
nSV = 2491, nBSV = 853
......*...*
optimization finished, #iter = 9700
obj = -1260.480464, rho = -0.322423
nSV = 3845, nBSV = 1587
Total nSV = 3845
......*...*
optimization finished, #iter = 9950
obj = -1239.615416, rho = -0.348236
nSV = 3873, nBSV = 1587
Total nSV = 3873
......*....*
optimization finished, #i

### Approach 2: word embedding

In [47]:
_ = SVM(X_train_vectors, y_train, X_test_vectors, y_test)

[LibSVM].....*..*.*
optimization finished, #iter = 8662
obj = -2071.714389, rho = 9.103812
nSV = 2230, nBSV = 1857
Total nSV = 2230
.....*...*..*
optimization finished, #iter = 9492
obj = -2081.898901, rho = 8.840646
nSV = 2247, nBSV = 1510
Total nSV = 2247
.....*..*
optimization finished, #iter = 7056
obj = -2074.884539, rho = 9.583814
nSV = 2240, nBSV = 1522
Total nSV = 2240
....*..*
optimization finished, #iter = 6744
obj = -2071.024865, rho = 9.061015
nSV = 2237, nBSV = 1889
Total nSV = 2237
....*.*.*
optimization finished, #iter = 5761
obj = -2056.512079, rho = 9.249406
nSV = 2209, nBSV = 1884
Total nSV = 2209
.....*...*.*
optimization finished, #iter = 8269
obj = -2500.837423, rho = -9.375802
nSV = 2647, nBSV = 2043
.........*...*.....*
optimization finished, #iter = 17350
obj = -2197.209990, rho = -0.483711
nSV = 3415, nBSV = 2642
Total nSV = 3415
.......*...*.....*
optimization finished, #iter = 15184
obj = -2159.869950, rho = -0.647812
nSV = 3372, nBSV = 2576
Total nSV = 3372


### Approach 3: tfidf + word embedding

In [48]:
from scipy.sparse import hstack

def add_glove_to_tfidf(tfidf_matrix, glove_embedding_matrix):
    enhanced_tfidf_matrix = hstack([tfidf_matrix, glove_embedding_matrix])
    return enhanced_tfidf_matrix

X_train_tfidf_glove = add_glove_to_tfidf(X_train_clean_tfidf, X_train_vectors)
X_test_tfidf_glove = add_glove_to_tfidf(X_test_clean_tfidf, X_test_vectors)
_ = SVM(X_train_tfidf_glove, y_train, X_test_tfidf_glove, y_test)

[LibSVM].....*..*
optimization finished, #iter = 7653
obj = -528.536886, rho = 2.734790
nSV = 1667, nBSV = 120
Total nSV = 1667
.....*...*
optimization finished, #iter = 8246
obj = -550.041230, rho = 2.935912
nSV = 1684, nBSV = 194
Total nSV = 1684
.....*..*
optimization finished, #iter = 7584
obj = -523.319336, rho = 3.747095
nSV = 1647, nBSV = 112
Total nSV = 1647
.....*...*
optimization finished, #iter = 8059
obj = -541.124638, rho = 3.047655
nSV = 1741, nBSV = 148
Total nSV = 1741
.....*...*
optimization finished, #iter = 8111
obj = -526.220052, rho = 2.953224
nSV = 1694, nBSV = 122
Total nSV = 1694
......*..*
optimization finished, #iter = 8921
obj = -620.553904, rho = -2.968800
nSV = 1898, nBSV = 521
........*....*
optimization finished, #iter = 12196
obj = -740.457620, rho = -1.682736
nSV = 2510, nBSV = 859
Total nSV = 2510
........*....*
optimization finished, #iter = 12033
obj = -738.109092, rho = -1.563510
nSV = 2498, nBSV = 836
Total nSV = 2498
.......*....*
optimization fin

### Additional Features

- Sentence Length 
- Count Booster Words


In [22]:
# Count sentence length
# Count of booster words (with preprocessing)

import pandas as pd
import numpy as np
import re
import nltk
from scipy.sparse import hstack
import time

# Uncomment if you have not downloaded the 'punkt' and 'averaged_perceptron_tagger' packages
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')

def add_custom_feature_to_tfidf(data, tfidf_matrix, feature_extraction_func):
    custom_feature = data.apply(feature_extraction_func)
    custom_feature_sparse = np.array(custom_feature).reshape(-1, 1)
    enhanced_tfidf_matrix = hstack([tfidf_matrix, custom_feature_sparse])
    return enhanced_tfidf_matrix

# Feature extraction functions
def count_sentence_length(text):
    return len(nltk.sent_tokenize(text))

def count_quotations(text):
    return text.count('"')

def count_numbers(text):
    return len(re.findall(r'\d+', text))

def count_proper_nouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    proper_nouns = [word for word, pos in tagged if pos == 'NNP']
    return len(proper_nouns)

def count_conjunctions(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    conjunctions = [word for word, pos in tagged if pos == 'CC']
    return len(conjunctions)

def count_superlatives(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    superlatives = [word for word, pos in tagged if pos == 'JJS']
    return len(superlatives)

def count_1st_pronouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    first_pronouns = [word for word, pos in tagged if pos == 'PRP' and word.lower() in ['i', 'me', 'my', 'mine']]
    return len(first_pronouns)

def count_3rd_pronouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    third_pronouns = [word for word, pos in tagged if pos == 'PRP' and word.lower() in 
                      ['he', 'him', 'his', 'she', 'her', 'hers', 'it', 'its', 'they', 'them', 'their', 'theirs']]
    return len(third_pronouns)

def count_booster_words(text):
    booster_words = """absolutely amazingly awfully completely considerably definitely entirely exceedingly
                    extremely highly incredibly indeed quite really remarkably so terribly totally truly utterly very"""
    booster_words = booster_words.split(" ")
    return len([word for word in text.split(" ") if word in booster_words])

# Evaluate the performance of the model with the custom features
# For the purpose of comparison, we can just use the logistic regression model across all the custom features
features_methods_dictionary =  {
    "Sentence Length": count_sentence_length, #ok
    # "Count Quotations": count_quotations,
    # "Count Numbers": count_numbers,
    # "Count Proper Nouns": count_proper_nouns,
    # "Count Conjunctions": count_conjunctions,
    # "Count Superlatives": count_superlatives,
    # "Count 1st Pronouns": count_1st_pronouns,
    # "Count 3rd Pronouns": count_3rd_pronouns,
    "Count Booster Words": count_booster_words # ok
}

training_datasets = {
    # "Not Processed": [X_train, X_train_tfidf, X_test, X_test_tfidf],
    "Processed": [X_train_clean, X_train_clean_tfidf, X_test_clean, X_test_clean_tfidf]
}
for dataset_name , datasets in training_datasets.items():
    X_train = datasets[0]
    X_train_tfidf = datasets[1]
    X_test = datasets[2]
    X_test_tfidf = datasets[3]
    if not (isinstance(X_train, pd.Series) or isinstance(X_train, pd.DataFrame)):
        X_train = pd.Series(X_train)
    if not (isinstance(X_test, pd.Series) or isinstance(X_test, pd.DataFrame)):
        X_test = pd.Series(X_test)
    for feature_name, feature_matrix in features_methods_dictionary.items():
        time_start = time.time()
        if dataset_name == "Not Processed":
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        else:
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        _ = SVM(X_train_enhanced, y_train, X_test_enhanced, y_test)
        time_end = time.time()
        print(f"Time taken for {feature_name} on {dataset_name} dataset: {time_end - time_start} seconds")

[nltk_data] Downloading package punkt to /Users/stella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[LibSVM]....*....*.*
optimization finished, #iter = 9456
obj = -790.273763, rho = 1.006198
nSV = 2245, nBSV = 454
Total nSV = 2245
......*..*
optimization finished, #iter = 8024
obj = -799.318501, rho = 0.965214
nSV = 2231, nBSV = 354
Total nSV = 2231
......*..*
optimization finished, #iter = 8013
obj = -767.939409, rho = 0.994926
nSV = 2200, nBSV = 391
Total nSV = 2200
.....*..*
optimization finished, #iter = 7940
obj = -787.390706, rho = 1.050144
nSV = 2237, nBSV = 329
Total nSV = 2237
......*..*
optimization finished, #iter = 8076
obj = -785.092674, rho = 1.008163
nSV = 2229, nBSV = 321
Total nSV = 2229
......*....*.*
optimization finished, #iter = 10565
obj = -903.863326, rho = -1.027343
nSV = 2470, nBSV = 832
.......*...*
optimization finished, #iter = 10564
obj = -1243.698809, rho = -0.130755
nSV = 3880, nBSV = 1423
Total nSV = 3880
.......*...*
optimization finished, #iter = 10925
obj = -1254.426610, rho = -0.144228
nSV = 3883, nBSV = 1459
Total nSV = 3883
.......*...*
optimizat

## tfidf + word embeddings + sentence length + booster words count

In [57]:
import copy
def add_custom_feature_to_vectors(data, vector_matrix, methods_dictionary):
    temp_matrix = copy.deepcopy(vector_matrix)
    for _, feature_func in methods_dictionary.items():
        custom_feature = data.apply(feature_func)
        custom_feature_sparse = np.array(custom_feature).reshape(-1, 1)
        temp_matrix = hstack([temp_matrix, custom_feature_sparse])
    return temp_matrix

optimal_features_methods_dictionary =  {
    "Sentence Length": count_sentence_length, 
    "Count Booster Words": count_booster_words 
}

training_datasets = {
    # "Not Processed": [X_train, X_train_tfidf_glove, X_test, X_test_tfidf_glove],
    "Processed": [X_train_clean, X_train_tfidf_glove, X_test_clean, X_test_tfidf_glove]
}
for dataset_name , datasets in training_datasets.items():
    X_train = datasets[0]
    X_train_tfidf = datasets[1]
    X_test = datasets[2]
    X_test_tfidf = datasets[3]
    if not (isinstance(X_train, pd.Series) or isinstance(X_train, pd.DataFrame)):
        X_train = pd.Series(X_train)
    if not (isinstance(X_test, pd.Series) or isinstance(X_test, pd.DataFrame)):
        X_test = pd.Series(X_test)
    if not (isinstance(X_train_clean, pd.Series) or isinstance(X_train_clean, pd.DataFrame)):
        X_train_clean = pd.Series(X_train_clean)
    if not (isinstance(X_test_clean, pd.Series) or isinstance(X_test_clean, pd.DataFrame)):
        X_test_clean = pd.Series(X_test_clean)

    time_start = time.time()
    if dataset_name == "Not Processed":
        X_train_enhanced = add_custom_feature_to_vectors(X_train, X_train_tfidf_glove, optimal_features_methods_dictionary)
        X_test_enhanced = add_custom_feature_to_vectors(X_test, X_test_tfidf_glove, optimal_features_methods_dictionary)
    else:
        X_train_enhanced = add_custom_feature_to_vectors(X_train_clean, X_train_tfidf_glove, optimal_features_methods_dictionary)
        X_test_enhanced = add_custom_feature_to_vectors(X_test_clean, X_test_tfidf_glove, optimal_features_methods_dictionary)
    _ = SVM(X_train_enhanced, y_train, X_test_enhanced, y_test)
    time_end = time.time()
    print(f"Time taken on {dataset_name} dataset: {time_end - time_start} seconds")

SVM Accuracy Score ->  71.2
              precision    recall  f1-score   support

           0       0.87      0.79      0.83       750
           1       0.75      0.47      0.58       750
           2       0.55      0.64      0.59       750
           3       0.73      0.94      0.82       750

    accuracy                           0.71      3000
   macro avg       0.72      0.71      0.71      3000
weighted avg       0.72      0.71      0.71      3000

Confusion Matrix:
[[596 107  30  17]
 [ 28 353 348  21]
 [ 40   5 479 226]
 [ 21   5  16 708]]
Time taken on Processed dataset: 2949.589693069458 seconds
